## License

This software component is licensed by ST under BSD-3-Clause license,
the "License"; 

You may not use this file except in compliance with the
License. 

You may obtain a copy of the License at: https://opensource.org/licenses/BSD-3-Clause

Copyright (c) 2023 STMicroelectronics. All rights reserved

# Creating a tltb model for Mobilenet v2 using Pytorch
This notebook presents scripts to take a pretarained Mobilenet v2 model and export it as onnx and a tao model. 
In the following sections you can see:
* [how can you start from your own model in PyTorch, load the weights to it from a public repo and export it as an ONNX model](#head-1), 
* [convert it to tltb model which you can to bring to TAO framework](#head-2). 

# # Creating a tltb model for mobilenetv2 using Pytorch <a class="anchor" id="head-1"></a>
Installing all the dependencies

In [ ]:
!pip install efficientnet-pytorch==0.6.3 
!pip install segmentation-models-pytorch==0.2.1 
!pip install timm torch==1.10.1
!pip install torchvision==0.11.2
!pip install tqdm==4.64.0
!pip install netron
!pip install tensorflow==2.9.1
!pip install nvidia-tao-byom
!pip install numpy==1.23.4

In [ ]:
# import the required dependecies
import torch
import sys
sys.path.append('./mobilenetv2_utils/')
from mobilenetv2 import mobilenetv2

# variables to be used during onnx model creation
model_name = "mobilenet_v2"
ALPHA = 0.5
resolution = 128
pretrained_weights = "./mobilenetv2_utils/mobilenetv2_0.5-eaa6f9ad.pth"

''' 
 Create a model using the mobilenetv2.py file and load the weights. 
 The script "mobilenetv2.py" and weights "mobilenetv2_0.5-eaa6f9ad.pth"
 can be obtained from https://github.com/d-li14/mobilenetv2.pytorch.
'''
model = mobilenetv2(width_mult=ALPHA)
model.load_state_dict(torch.load(pretrained_weights))

'''
alternatively one can create the model using torch vision api directly as below
model = torch.hub.load('pytorch/vision:v0.10.0', torch_model_name, width_mult=ALPHA, pretrained=pretrained_weights)
'''


# export the model as onnx
dummy_input = torch.randn(1, 3, resolution, resolution, requires_grad=True) 

# this is needed as inpot to onnx.export
export_name = f"{model_name}_{resolution}_{str(ALPHA).replace('.', '_')}.onnx"

# exporting a torch model as .onnx
torch.onnx.export(model, dummy_input, export_name, input_names=["input_1"], verbose=False,
                  training="TrainingMode.TRAIN", do_constant_folding=False)

## 1.1: View the network using netron

In [ ]:
import netron
netron.start(export_name)

# 2. Convert .onnx to .tltb with TAO-Byom<a class="anchor" id="head-2"></a>
This step is to create the .tltb file from the .onnx file. .tltb is an internal format of TAO and this step is needed to start training using TAO. This step will create a byom_model directory with the file and complementary information for this in it.

If we wish to fine tune the pretrained model (as is the case) on a different dataset through TAO Toolkit, we must remove the classification head for ImageNet. Hence, the final converted TAO model should only contain layers up to the penultimate layer, which is a layer before the average pooling. In this case, the node name is `463`. Adding `-p 463` argument, removes the head of the model while creatig the `.tltb` file.

In [ ]:
%env TF_ENABLE_ONEDNN_OPTS=1
# creating the .tltb file
!tao_byom -m {export_name} -r ../pretrained_mobilenetv2/mobilenetv2_128_0_5 -n mobilenetv2_128_0_5 -k nvidia_tlt -p 463